In [82]:
DATA_PATH=r'..\data\processed\new_data.pickle'
MLFLOW_TRACKING_URI="..\models\mlruns"
MLFLOW_EXPERIMENT_NAME = "Amazon_products_recommendation_system"
LOG_PATH = "../models/temp/"
LOG_DATA_PKL    =  "data.pkl"
LOG_MODEL_PKL   =  "model.pkl"
LOG_METRICS_PKL =  "metrics.pkl"

In [83]:
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
# import mlflow
# from mlflow import MlflowClient
import pickle
import torch
from torch import nn
import torch.optim as optim
import sklearn


In [84]:
df=pd.read_pickle(DATA_PATH)
df

,User_ID,Product_ID,Ratings,year,month
1309,A3LDPF5FMB782Z,1400501466,5.0,2012,5
1321,A1A5KUIIIHFF4U,1400501466,1.0,2012,3
1334,A2XIOXRRYX0KZY,1400501466,3.0,2013,6
1450,AW3LX47IHPFRL,1400501466,5.0,2012,6
1455,A1E3OB6QMBKRYZ,1400501466,1.0,2012,10
...,...,...,...,...,...
7824422,A34BZM6S9L7QI4,B00LGQ6HL8,5.0,2014,7
7824423,A1G650TTTHEAL5,B00LGQ6HL8,5.0,2014,7
7824424,A25C2M3QF9G7OQ,B00LGQ6HL8,5.0,2014,7
7824425,A1E1LEVQ9VQNK,B00LGQ6HL8,5.0,2014,7


In [85]:
user_item=df.pivot(index='User_ID',columns='Product_ID',values='Ratings').fillna(0)
user_item

Product_ID,1400501466,1400532655,9983891212,B00000DM9W,B00000J1V5,B00000JDF5,B00000JDF6,B00000K135,B00000K4KH,B00001P4XA,...,B00KNM763E,B00KONCDVM,B00KVNY2KA,B00KXAFYZS,B00KYMCJF8,B00L21HC7A,B00L2442H0,B00L26YDA4,B00L3YHF6O,B00LGQ6HL8
User_ID,,,,,,,,,,,,,,,,,,,,,
A100UD67AHFODS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A100WO06OQR8BQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A105S56ODHGJEK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A105TOJ6LTVMBG,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10AFVU66A79Y1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZBXKUH4AIW3X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AZCE11PSTCH1L,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AZMY6E8B52L2T,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,5.0,0.0,5.0,5.0,5.0,0.0,0.0


In [86]:
user_item.shape

(1466, 5523)

In [87]:
from sklearn.decomposition import TruncatedSVD

# Reducing to 100 latent dimensions
svd = TruncatedSVD(n_components=100)
user_embeddings = svd.fit_transform(user_item)

In [99]:
user_embeddings

array([[ 3.53330541,  4.40360228, -1.53052736, ...,  1.18090294,
         0.03301559,  0.05778452],
       [ 6.94130631, -1.92579534, -1.67357295, ..., -1.54670264,
        -1.3552569 , -0.44802601],
       [ 6.34439439, -4.10044608,  2.6961316 , ...,  1.15551302,
        -0.48327295,  1.51746706],
       ...,
       [ 9.6277111 , 17.81652055, -0.36660201, ..., -0.20895098,
        -1.74434536,  4.81743923],
       [ 2.47535058, -0.32131661, -0.82123224, ...,  1.77949437,
         0.3649639 , -1.45024398],
       [ 0.93023304,  0.43188796, -1.2040544 , ...,  0.48162532,
        -0.50897649,  0.55024604]])

In [100]:
svd.components_

array([[ 0.00096706,  0.00092971,  0.00316971, ...,  0.0194871 ,
         0.03023553,  0.00997189],
       [-0.00052815,  0.00111599,  0.0012699 , ...,  0.05146367,
         0.04333964,  0.03368475],
       [-0.00060992, -0.00214826, -0.00825974, ...,  0.02118831,
         0.01503552,  0.01637248],
       ...,
       [-0.00169206,  0.0020079 , -0.00643174, ..., -0.01054749,
        -0.02524288, -0.00094783],
       [ 0.00042355, -0.0005518 , -0.00919848, ..., -0.00032323,
         0.00805532, -0.00407286],
       [-0.00315683, -0.00498021, -0.00096713, ..., -0.0078656 ,
        -0.01891403,  0.01231645]])

In [105]:
# pd.DataFrame(user_embeddings@svd.components_)

In [88]:
user_embeddings
svd.components_.shape

(100, 5523)

In [144]:
class Generator(nn.Module):
    def __init__(self,latent_dim,profile_dim):
        super(Generator,self).__init__()
        self.model=nn.Sequential(
            nn.Linear(latent_dim,128),
            nn.ReLU(),
            nn.Linear(128,256),
            nn.ReLU(),
            nn.Linear(256,512),
            nn.ReLU(),
            nn.Linear(512,profile_dim),
            nn.Sigmoid()
        )

    def forward (self,z) :
        return self.model(z)   
    
class Discriminator(nn.Module):
    def __init__(self,profile_dim):
        super(Discriminator,self).__init__()
        self.model=nn.Sequential(

            nn.Linear(profile_dim,256),
            nn.LeakyReLU(.2),
            nn.Linear(256,128),
            nn.LeakyReLU(.2),
            nn.Linear(128,1),
            nn.Sigmoid()
        )    

    def forward(self,profile):
        return self.model(profile)    

In [145]:
latent_dim = 100  # Dimension of random noise
profile_dim = 5523  # Profile feature dimension
batch_size = 64
learning_rate = 0.0002

In [146]:
generator=Generator(latent_dim,profile_dim)
discrimnator=Discriminator(profile_dim)

criterion=nn.BCELoss()
optimizer_G=optim.Adam(generator.parameters(),lr=learning_rate)
optimizer_D=optim.Adam(discrimnator.parameters(),lr=learning_rate)




In [147]:
# real_profiles=torch.tensor(user_embeddings)
# real_profiles.size(0)

In [155]:
num_epochs=1000
for epoch in range (num_epochs):
    optimizer_D.zero_grad()
    real_profiles = torch.tensor(user_item.values, dtype=torch.float32)  # Shape: (num_users, profile_dim)

    real_profiles=torch.tensor(real_profiles)
    real_lapels=torch.ones(real_profiles.size(0),1)
    real_outputs=discrimnator(real_profiles)
    d_loss_real=criterion(real_outputs,real_lapels)

    z=torch.randn(real_profiles.size(0),latent_dim)
    fake_profiles=generator(z)
    fake_labels=torch.zeros(real_profiles.size(0),1)
    fake_outputs=discrimnator(fake_profiles)
    d_loss_fake=criterion(fake_outputs,fake_labels)

    d_loss=d_loss_fake+d_loss_real
    d_loss.backward(retain_graph=True)
    optimizer_D.step()


    optimizer_G.zero_grad()
    fake_outputs=discrimnator( fake_profiles)
    # print(fake_outputs)
    g_loss=criterion(fake_outputs,real_lapels)
    g_loss.backward()
    optimizer_G.step()
    if epoch % 100 == 0:
        print(f"Epoch [{epoch}/{num_epochs}], d_loss: {d_loss.item()}, g_loss: {g_loss.item()}")




C:\Users\Mohamed Mosaad\AppData\Local\Temp\ipykernel_17560\3866072594.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  real_profiles=torch.tensor(real_profiles)


Epoch [0/1000], d_loss: 0.014634043909609318, g_loss: 8.627960205078125
Epoch [100/1000], d_loss: 0.004645331297069788, g_loss: 8.456727027893066
Epoch [200/1000], d_loss: 0.00338601297698915, g_loss: 8.714179992675781
Epoch [300/1000], d_loss: 0.03172688186168671, g_loss: 13.886958122253418
Epoch [400/1000], d_loss: 0.0019262657733634114, g_loss: 11.01513385772705
Epoch [500/1000], d_loss: 0.006052048411220312, g_loss: 9.920281410217285
Epoch [600/1000], d_loss: 0.0032842871733009815, g_loss: 7.094478130340576
Epoch [700/1000], d_loss: 0.0014289512764662504, g_loss: 8.171209335327148
Epoch [800/1000], d_loss: 0.0009938082657754421, g_loss: 9.358089447021484
Epoch [900/1000], d_loss: 0.000467195495730266, g_loss: 11.117900848388672


In [156]:
z=torch.randn(20,100)
synthetic_profiles=generator(z)
scaled_profiles = synthetic_profiles * 5  # Map [0, 1] to [0, 5]
discrete_profiles = torch.round(scaled_profiles)

# Threshold low values to 0 if needed
threshold = 1
discrete_profiles[discrete_profiles < threshold] = 0
pd.DataFrame(torch.detach(discrete_profiles))


,0,1,2,3,4,5,6,7,8,9,...,5513,5514,5515,5516,5517,5518,5519,5520,5521,5522
0,0.0,5.0,5.0,5.0,0.0,0.0,0.0,3.0,0.0,2.0,...,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
1,0.0,5.0,5.0,5.0,0.0,0.0,0.0,3.0,0.0,2.0,...,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
2,0.0,5.0,5.0,5.0,0.0,0.0,0.0,3.0,0.0,2.0,...,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
3,0.0,5.0,5.0,5.0,0.0,0.0,0.0,3.0,0.0,2.0,...,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
4,0.0,5.0,5.0,5.0,0.0,0.0,0.0,3.0,0.0,2.0,...,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
5,0.0,5.0,5.0,5.0,0.0,0.0,0.0,3.0,0.0,2.0,...,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
6,0.0,5.0,5.0,5.0,0.0,0.0,0.0,3.0,0.0,2.0,...,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
7,0.0,5.0,5.0,5.0,0.0,0.0,0.0,3.0,0.0,2.0,...,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
8,0.0,5.0,5.0,5.0,0.0,0.0,0.0,3.0,0.0,2.0,...,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
9,0.0,5.0,5.0,5.0,0.0,0.0,0.0,3.0,0.0,2.0,...,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0


In [143]:
scaled_profiles = (synthetic_profiles + 1) 

discrete_profiles = torch.round(scaled_profiles)
threshold = 1
discrete_profiles[discrete_profiles < threshold] = 0
scaled_profiles
# # Convert to a DataFrame for easier handling, with product IDs as columns
# synthetic_profiles_df = pd.DataFrame(
#     discrete_profiles.detach().numpy(), 
#     columns=[f"Product_{i}" for i in range(profile_dim)]
# )
# synthetic_profiles_df


tensor([[1.1582, 1.0201, 1.0195,  ..., 0.9646, 1.0073, 1.0522],
        [1.1478, 1.0322, 1.0232,  ..., 0.9743, 1.0116, 1.0559],
        [1.1606, 1.0421, 1.0299,  ..., 0.9753, 1.0112, 1.0558],
        ...,
        [1.1548, 1.0279, 1.0248,  ..., 0.9739, 1.0047, 1.0492],
        [1.1370, 1.0273, 1.0242,  ..., 0.9839, 1.0071, 1.0480],
        [1.0858, 1.0215, 1.0081,  ..., 1.0058, 1.0080, 1.0354]],
       grad_fn=<AddBackward0>)

In [98]:
# z=torch.randn(10,100)
# synthetic_profiles=generator(z)
# original_dim_synthetic_profiles = synthetic_profiles @ torch.tensor(svd.components_, dtype=torch.float32)
# original_dim_synthetic_profiles = original_dim_synthetic_profiles.detach().numpy()
# scaled_profiles = (original_dim_synthetic_profiles + 1) * 2 # Maps from [-1, 1] to [1, 5]
# threshold = 1
# scaled_profiles[scaled_profiles < threshold] = 0
# synthetic_profiles_df = pd.DataFrame(scaled_profiles, columns=[f"Product_{i}" for i in range(product_embeddings.shape[1])])
# synthetic_profiles_df

,Product_0,Product_1,Product_2,Product_3,Product_4,Product_5,Product_6,Product_7,Product_8,Product_9,...,Product_5513,Product_5514,Product_5515,Product_5516,Product_5517,Product_5518,Product_5519,Product_5520,Product_5521,Product_5522
0,2.500146,2.483446,2.545823,2.430334,2.418919,2.560656,2.573986,2.335510,2.598732,2.702518,...,2.423273,2.607817,2.725234,2.447518,2.378659,2.194475,2.133805,2.163052,2.176616,2.261167
1,2.499887,2.491398,2.550073,2.457689,2.431810,2.535469,2.550894,2.375421,2.583809,2.640193,...,2.455241,2.574931,2.679621,2.489161,2.409468,2.319759,2.261487,2.276363,2.296059,2.308084
2,2.501173,2.499612,2.542211,2.478822,2.458534,2.520449,2.532574,2.424906,2.574032,2.589723,...,2.480690,2.550498,2.616248,2.502605,2.441787,2.390606,2.357120,2.358881,2.381980,2.369528
3,2.498349,2.495599,2.544780,2.485741,2.461424,2.529714,2.545595,2.413802,2.555633,2.597207,...,2.473763,2.547965,2.631993,2.505312,2.443451,2.412105,2.353026,2.371088,2.388363,2.356236
4,2.500267,2.501994,2.526376,2.500614,2.485562,2.520482,2.522143,2.472629,2.543218,2.556836,...,2.498905,2.526371,2.564127,2.508245,2.482816,2.474928,2.453501,2.452916,2.480318,2.416341
5,2.499891,2.501807,2.535126,2.494161,2.487670,2.521840,2.527334,2.471783,2.548614,2.571597,...,2.493630,2.527498,2.582798,2.511891,2.470459,2.461424,2.435823,2.438302,2.471032,2.404418
6,2.499861,2.500766,2.527890,2.504906,2.499042,2.527144,2.529876,2.480635,2.533073,2.563903,...,2.500032,2.526649,2.567344,2.513847,2.485996,2.490911,2.462953,2.467316,2.493640,2.420988
7,2.500702,2.502642,2.527015,2.501036,2.490412,2.520514,2.523102,2.477982,2.544531,2.554900,...,2.500430,2.525951,2.560450,2.510499,2.479122,2.477944,2.457366,2.457349,2.483157,2.417714
8,2.497859,2.490908,2.552183,2.464734,2.440772,2.537108,2.552458,2.385446,2.570580,2.637084,...,2.450376,2.561986,2.678980,2.493556,2.414977,2.342915,2.284178,2.301177,2.324464,2.306963
9,2.501272,2.502552,2.527914,2.497245,2.485324,2.520929,2.523123,2.473194,2.547755,2.557995,...,2.499270,2.529712,2.568072,2.510451,2.474978,2.469494,2.449777,2.447256,2.476366,2.412649


In [180]:

input_dim = 5523   # Product dimension (number of products)
latent_dim = 64    # Latent space dimension for VAE
batch_size = 32
num_epochs = 100
learning_rate = 0.001

In [181]:
class VAEGenerator(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(VAEGenerator, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, latent_dim * 2)  # For both mean and log-variance
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, input_dim),
            nn.Sigmoid()  # Ratings scaled between 0 and 1
        )

    def encode(self, x):
        h = self.encoder(x)
        mu, log_var = h[:, :latent_dim], h[:, latent_dim:]
        return mu, log_var

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        return self.decoder(z), mu, log_var

# Define the Discriminator
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)


In [182]:
vae_generator = VAEGenerator(input_dim, latent_dim)
discriminator = Discriminator(input_dim)

In [183]:

vae_optimizer = optim.Adam(vae_generator.parameters(), lr=learning_rate)
generator_optimizer = optim.Adam(vae_generator.decoder.parameters(), lr=learning_rate)  # Separate optimizer for generator

discriminator_optimizer = optim.Adam(discriminator.parameters(), lr=learning_rate)

In [184]:
reconstruction_criterion = nn.MSELoss()
adversarial_criterion = nn.BCELoss()

In [185]:
from torch.utils.data import DataLoader, TensorDataset

user_product_matrix = torch.tensor(user_item.values,dtype=torch.float32)
dataset = TensorDataset(user_product_matrix)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [187]:
import torch

# Enable anomaly detection for debugging
torch.autograd.set_detect_anomaly(True)

# Training loop
for epoch in range(num_epochs):
    for i, (batch,) in enumerate(dataloader):
        batch = batch.squeeze(1)
        
        # ================== Train VAE (Encoder-Decoder) ==================
        # Generate profiles from the VAE
        generated_profile, mu, log_var = vae_generator(batch)

        # Create masks to avoid in-place operations
        mask = (batch != 0).clone()  # Mask for valid ratings

        # Ensure the tensors used for loss calculations are not modified in place
        masked_generated_profile = generated_profile * mask
        masked_batch = batch * mask

        # Compute the reconstruction loss
        reconstruction_loss = reconstruction_criterion(masked_generated_profile.clone(), masked_batch.clone())

        # KL Divergence
        kl_divergence = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp()) / batch.size(0)

        # Total VAE loss
        vae_loss = reconstruction_loss + kl_divergence
        vae_optimizer.zero_grad()
        vae_loss.backward(retain_graph=True)  # Retain the graph for the next backward pass
        vae_optimizer.step()

        # ================== Train Discriminator ==================
        real_labels = torch.ones(batch.size(0), 1)
        fake_labels = torch.zeros(batch.size(0), 1)

        # Discriminator loss on real profiles
        real_loss = adversarial_criterion(discriminator(batch.detach()), real_labels)

        # Discriminator loss on generated (fake) profiles
        fake_loss = adversarial_criterion(discriminator(generated_profile.detach()), fake_labels)
        
        # Total discriminator loss
        discriminator_loss = real_loss + fake_loss
        discriminator_optimizer.zero_grad()
        discriminator_loss.backward()
        discriminator_optimizer.step()

        # ================== Train Generator (VAE Decoder) ==================
        # Clone generated_profile to avoid in-place operations
        cloned_generated_profile = generated_profile.clone()  # Clone here
        generator_loss = adversarial_criterion(discriminator(cloned_generated_profile), real_labels)
        generator_optimizer.zero_grad()
        generator_loss.backward()  # Ensure the computation graph is clean
        generator_optimizer.step()

    print(f"Epoch [{epoch + 1}/{num_epochs}], VAE Loss: {vae_loss.item()}, D Loss: {discriminator_loss.item()}, G Loss: {generator_loss.item()}")


c:\Users\Mohamed Mosaad\anaconda3\envs\pytorch_env\lib\site-packages\torch\autograd\graph.py:769: UserWarning: Error detected in AddmmBackward0. Traceback of forward call that caused the error:
  File "c:\Users\Mohamed Mosaad\anaconda3\envs\pytorch_env\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\Mohamed Mosaad\anaconda3\envs\pytorch_env\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "c:\Users\Mohamed Mosaad\anaconda3\envs\pytorch_env\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\Mohamed Mosaad\anaconda3\envs\pytorch_env\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\Mohamed Mosaad\anaconda3\envs\pytorch_env\lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "c:\Users\Mohamed Mosaad\anaconda3\envs\pytorch_env\lib\site-packa

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [256, 5523]], which is output 0 of AsStridedBackward0, is at version 3; expected version 2 instead. Hint: the backtrace further above shows the operation that failed to compute its gradient. The variable in question was changed in there or anywhere later. Good luck!

In [189]:
latent_dim = 100
num_users = 1466
num_products = 5523
batch_size = 128
epochs = 500

In [212]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(True),
            nn.Linear(256, num_products),
            nn.ReLU()
        )

    def forward(self, x):
        return self.model(x)

# Discriminator model
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(num_products, 256),
            nn.ReLU(True),
            nn.Dropout(0.4),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)


In [213]:
generator = Generator()
discriminator = Discriminator()

In [214]:
criterion = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=0.0002)
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002)

In [215]:
def train():
    real_profiles = torch.tensor(user_item.values,dtype=torch.float32) # Replace with your actual data
    for epoch in range(epochs):
        for i in range(0, num_users, batch_size):
            real_data = real_profiles[i:i+batch_size]
            batch_size_actual = real_data.size(0)
            
            # Train Discriminator
            noise = torch.randn(batch_size_actual, latent_dim)
            fake_data = generator(noise)
            real_labels = torch.ones(batch_size_actual, 1)
            fake_labels = torch.zeros(batch_size_actual, 1)
            outputs_real = discriminator(real_data)
            outputs_fake = discriminator(fake_data)
            d_loss_real = criterion(outputs_real, real_labels)
            d_loss_fake = criterion(outputs_fake, fake_labels)
            d_loss = d_loss_real + d_loss_fake
            optimizer_D.zero_grad()
            d_loss.backward()
            optimizer_D.step()
            
            # Train Generator
            noise = torch.randn(batch_size_actual, latent_dim)
            fake_data = generator(noise)
            outputs = discriminator(fake_data)
            g_loss = criterion(outputs, real_labels)
            optimizer_G.zero_grad()
            g_loss.backward()
            optimizer_G.step()
        
        if epoch % 100 == 0:
            print(f"{epoch}/{epochs} [D loss: {d_loss.item()}] [G loss: {g_loss.item()}]")

# Train the GAN
train()

0/1000 [D loss: 0.94769287109375] [G loss: 1.817845344543457]
100/1000 [D loss: 0.31211620569229126] [G loss: 1.466598629951477]
200/1000 [D loss: 0.046661920845508575] [G loss: 4.27742338180542]
300/1000 [D loss: 0.019628623500466347] [G loss: 4.773027420043945]
400/1000 [D loss: 0.004255291074514389] [G loss: 5.7544097900390625]
500/1000 [D loss: 0.014859076589345932] [G loss: 5.991272926330566]
600/1000 [D loss: 0.00381445768289268] [G loss: 7.077122688293457]
700/1000 [D loss: 0.0013609391171485186] [G loss: 6.921176433563232]
800/1000 [D loss: 0.002675817348062992] [G loss: 6.356795787811279]
900/1000 [D loss: 0.0009479106520302594] [G loss: 6.762637615203857]


In [208]:
noise = torch.randn(num_users, latent_dim)
synthetic_profiles = generator(noise).detach().numpy()

In [209]:
synthetic_profiles = (synthetic_profiles - synthetic_profiles.min()) / (synthetic_profiles.max() - synthetic_profiles.min()) * 3 + 1
synthetic_df = pd.DataFrame(synthetic_profiles, columns=[f'Product_{i+1}' for i in range(num_products)])
synthetic_df['User_ID'] = [f'Synthetic_User_{i+1}' for i in range(num_users)]

In [210]:
synthetic_df

,Product_1,Product_2,Product_3,Product_4,Product_5,Product_6,Product_7,Product_8,Product_9,Product_10,...,Product_5515,Product_5516,Product_5517,Product_5518,Product_5519,Product_5520,Product_5521,Product_5522,Product_5523,User_ID
0,5.314103e-07,2.819219e-07,1.251384e-07,3.999458e-07,7.756937e-08,1.359688e-06,1.066595e-06,1.617725e-06,7.807677e-07,6.889765e-07,...,0.000081,4.946278e-07,6.981419e-08,3.860194e-05,4.503882e-08,3.806463e-07,6.922241e-06,2.881597e-06,2.305486e-10,Synthetic_User_1
1,8.092474e-08,6.471997e-08,1.349108e-09,6.644839e-08,1.938961e-09,7.811246e-08,5.560947e-07,1.227078e-07,1.898383e-08,1.676980e-06,...,0.000025,5.048454e-10,1.523311e-09,4.482919e-07,3.620554e-11,3.679526e-07,1.217089e-07,7.220895e-10,4.787342e-11,Synthetic_User_2
2,3.759703e-09,5.822406e-10,1.328673e-10,4.532034e-12,1.573813e-10,4.341150e-08,5.580370e-07,1.658891e-08,2.532320e-08,1.276538e-08,...,0.000006,7.944859e-04,3.886457e-08,1.452128e-04,6.502775e-03,2.571572e-06,1.459723e-05,1.987746e-02,2.281291e-11,Synthetic_User_3
3,8.373038e-08,5.185706e-08,1.529121e-09,9.170477e-09,2.282295e-08,2.235196e-07,1.994207e-07,6.612472e-07,4.341838e-07,3.656787e-07,...,0.000134,7.376718e-08,1.211395e-08,5.371261e-05,2.589847e-08,1.244156e-06,7.686333e-06,1.078753e-06,3.759486e-10,Synthetic_User_4
4,6.856317e-07,5.703386e-08,1.767792e-10,1.108913e-04,1.088164e-08,4.662554e-09,3.551229e-08,2.912249e-06,1.549095e-07,2.411328e-06,...,0.000002,2.701955e-17,9.478278e-11,2.375169e-11,2.365395e-18,7.379301e-10,1.668385e-08,3.816288e-17,4.008376e-14,Synthetic_User_5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1461,7.977912e-07,5.487592e-08,5.396907e-10,2.426345e-09,2.703705e-08,1.275132e-08,1.382912e-07,7.640645e-07,5.804250e-07,1.020193e-06,...,0.000004,2.241783e-08,1.714673e-08,2.898357e-05,1.739481e-07,3.533750e-07,1.024256e-05,1.178962e-06,1.239422e-11,Synthetic_User_1462
1462,2.235613e-08,3.889928e-08,1.305751e-09,1.066955e-09,4.586323e-08,1.953462e-08,8.294174e-07,1.303684e-07,5.482424e-08,2.060432e-07,...,0.000047,5.991798e-05,1.885705e-08,2.129597e-05,4.030798e-06,6.059844e-07,4.538910e-06,5.490053e-05,1.479544e-10,Synthetic_User_1463
1463,3.447666e-07,1.143016e-08,1.452509e-09,4.623503e-06,5.857008e-08,4.763463e-07,2.305799e-07,7.776520e-06,4.491118e-07,4.033870e-06,...,0.000014,5.755569e-11,3.733926e-09,2.261483e-07,7.044178e-12,3.372175e-07,1.019219e-06,3.334882e-11,6.766665e-11,Synthetic_User_1464
1464,1.425405e-06,1.326070e-06,7.631009e-09,7.783332e-05,2.422495e-07,2.560887e-06,4.762432e-06,3.281802e-05,4.866965e-07,1.132047e-05,...,0.000046,1.058451e-11,4.568850e-09,3.692127e-07,7.569746e-13,1.545094e-08,1.499307e-06,1.012033e-11,4.286489e-10,Synthetic_User_1465


In [211]:
synthetic_profiles = (synthetic_profiles+1)   # Scale to range [1, 4]
synthetic_df = pd.DataFrame(synthetic_profiles, columns=[f'Product_{i+1}' for i in range(num_products)])
synthetic_df['User_ID'] = [f'Synthetic_User_{i+1}' for i in range(num_users)]

# Introduce sparsity (set some ratings to zero)
sparsity_mask = np.random.rand(*synthetic_df.shape) < 0.8  # Adjust 0.8 for desired sparsity level
synthetic_df = synthetic_df * sparsity_mask
pd.DataFrame(synthetic_df)

,Product_1,Product_2,Product_3,Product_4,Product_5,Product_6,Product_7,Product_8,Product_9,Product_10,...,Product_5515,Product_5516,Product_5517,Product_5518,Product_5519,Product_5520,Product_5521,Product_5522,Product_5523,User_ID
0,1.000000,1.000000,1.0,0.000000,1.0,0.000000,0.000000,1.000002,1.000001,1.000001,...,1.000081,1.00000,1.0,0.000000,1.000000,1.000000,1.000007,1.000003,0.0,Synthetic_User_1
1,1.000000,1.000000,1.0,0.000000,1.0,0.000000,1.000001,1.000000,1.000000,1.000002,...,1.000025,0.00000,0.0,0.000000,1.000000,1.000000,1.000000,1.000000,1.0,Synthetic_User_2
2,1.000000,1.000000,1.0,0.000000,0.0,1.000000,0.000000,1.000000,1.000000,0.000000,...,1.000006,0.00000,1.0,1.000145,1.006503,1.000003,1.000015,1.019877,0.0,Synthetic_User_3
3,1.000000,1.000000,1.0,0.000000,1.0,1.000000,1.000000,1.000001,0.000000,1.000000,...,1.000134,1.00000,1.0,1.000054,1.000000,1.000001,1.000008,1.000001,1.0,Synthetic_User_4
4,1.000001,1.000000,1.0,1.000111,1.0,1.000000,0.000000,1.000003,0.000000,1.000002,...,1.000002,1.00000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1461,1.000001,1.000000,1.0,0.000000,1.0,1.000000,1.000000,1.000001,1.000001,1.000001,...,1.000004,1.00000,1.0,1.000029,1.000000,0.000000,0.000000,1.000001,1.0,
1462,1.000000,1.000000,1.0,1.000000,1.0,1.000000,1.000001,1.000000,1.000000,0.000000,...,1.000047,1.00006,1.0,1.000021,0.000000,1.000001,1.000005,1.000055,1.0,Synthetic_User_1463
1463,1.000000,1.000000,1.0,0.000000,1.0,1.000000,1.000000,1.000008,0.000000,1.000004,...,1.000014,1.00000,1.0,0.000000,1.000000,1.000000,1.000001,1.000000,1.0,Synthetic_User_1464
1464,0.000000,1.000001,1.0,1.000078,1.0,1.000003,1.000005,1.000033,0.000000,1.000011,...,1.000046,1.00000,1.0,1.000000,1.000000,1.000000,1.000002,1.000000,1.0,Synthetic_User_1465
